# Import the modules needed, defining paths, and funtions

In [ ]:
from basic_functions import *

print('Cheking list od packages need it')
for package_i in list_py_package:
    debug_missing_module(module_name=package_i)

def phoronix_benchmark_reader(file):
    filename = file.split('/')[-1][:-4]
    pattern_text = r'Flexible IO Tester - Type: (?P<type>.+) - Engine: (?P<engine>.+) - Buffered: (?P<buffered>.+) - Direct: (?P<direct>.+) - Block Size: (?P<block_size>.+)KB - Disk Target: (?P<disk_target>.+) .MB.s.'
    pattern_fio = re.compile(pattern_text)

    pattern_text = r'Stream - Type: (?P<type>.+) .MB.s.'
    pattern_stream = re.compile(pattern_text)

    fio_data = []
    stream_data = []
    header = []
    whitespace_counter = 0

    with open(file) as f:
        reader = csv.reader(f, delimiter=',')
        for i,row in enumerate(reader):
            if (i==0):
                header.append('======================================== '+ filename + ' ========================================')
            elif row==[]:
                whitespace_counter += 1
                if whitespace_counter<3:
                    header.append('  '.join(row))
                else:
                    break
            elif whitespace_counter<2:
                header.append('  '.join(row))
            elif pattern_fio.match(row[0]):
                header_end = True
                groups = pattern_fio.match(row[0]).groupdict()
                groups['result'] = row[2]
                fio_data.append(groups)
            elif pattern_stream.match(row[0]):
                header_end = True
                groups = pattern_stream.match(row[0]).groupdict()
                groups['result'] = row[2]
                stream_data.append(groups)
            

    fio_df = pd.DataFrame(fio_data)
    stream_df = pd.DataFrame(stream_data)
    stream_df['result'] = stream_df['result'].astype('float')
    
    return fio_df, stream_df, header

def phoronix_benchmark_plotter(fio_df, stream_df, header, output_dir):

    fio_df['block_size'] = fio_df['block_size'].astype('int')
    fio_df['result'] = fio_df['result'].astype('float')
    
    type_list = fio_df.type.unique()
    target_list = fio_df.disk_target.unique()

    fig, axs = plt.subplots(1, 4, figsize=(20,4))
    plt.style.use('default')
    
    for i, test in enumerate(type_list):
        for target in target_list:
            df_test_i = fio_df[(fio_df.type == test) & (fio_df.disk_target == target)]
            axs[i].plot(df_test_i.block_size, df_test_i.result, label=target)
        
        axs[i].set_title('Flexible IO Tester - ' + test)
        if i == 0:    
            axs[i].set_ylabel('IO speed (MB/s)')
        axs[i].set_xlabel('block size (KB)')
        axs[i].grid()

    plt.savefig('{}/fio_plots.png'.format(output_dir))
    plt.close()

def report_test_options(df, pdf):
    pdf.write(5,'Options: \n')
    
    for column in df.columns:
        if column == 'result':
            pass
        else:
            unique = df[column].unique()
            pdf.write(5, column)
            pdf.write(5, ': ')
            pdf.write(5, ', '.join(unique))
            pdf.write(5, '\n')
    pdf.write(5,'\n')

def create_report_benchmark(input_dir, output_dir):
    directory([input_dir, output_dir])
    
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font('Arial', 'B', 16)
    pdf.cell(40,10,'Benchmark Report')
    pdf.write(5,'\n\n')
    
    benchmark_list = make_name_list_benchmark(input_dir)

    for pag, file_list in enumerate(benchmark_list):

        fio_df, stream_df, header = phoronix_benchmark_reader(file='{}/{}.csv'.format(input_dir, file_list))
        phoronix_benchmark_plotter(fio_df.copy(), stream_df, header, output_dir)
        
        pdf.set_font('Arial', '', 8)
        for line in header:
            pdf.write(5, line)
            pdf.write(5,'\n')
        
        test_suite = 'dunedaq-srv-benchmark'
        pdf.write(5, 'Test suite name: {} \n'.format(test_suite))
        pdf.write(5, 'Suite definition and test versions here: performancetest/benchmark/local/{}/suite-definition.xml \n\n'.format(test_suite))
        
        pdf.set_font('Arial', 'B', 8)
        pdf.write(5, 'Flexible IO Tester')
        pdf.set_font('Arial', '', 8)
        pdf.write(5,' - tests read/write speed to RAID storage for SNB recording \n')
        report_test_options(fio_df, pdf)
        pdf.image('{}/fio_plots.png'.format(output_dir), w=200)
        pdf.write(5,'\n\n')
        
        pdf.set_font('Arial', 'B', 8)
        pdf.write(5, 'STREAM')
        pdf.set_font('Arial', '', 8)
        pdf.write(5, ' - tests CPU memory bandwidth in MB/s using 4 operation types \n')
        report_test_options(stream_df, pdf)
        pdf.write(5, stream_df.to_string())
        pdf.write(5,'\n\n')
        pdf.write(5, 'Average CPU memory bandwidth: {} MB/s \n'.format(float(stream_df.mean(numeric_only=True))))
        
        if pag+1 == len(benchmark_list):
            pass
        else:
            pdf.add_page()

    pdf.write(5,'\n\n')
    pdf.write(5,'The End, made on {}'.format(current_time()))
    pdf.output('{}/benchmark_report.pdf'.format(output_dir))
    
print('Ready to run and process')

# Benchmark report
Note: change the paths to fit yours

In [ ]:
# Phoronix data 
benchmark_path = '../benckmark_results'
report_path = '../reports'

create_report_benchmark(benchmark_path, report_path)

print('THE END')